# PHE Daily Cases

Created by Michael George (AKA Logiqx)

Website: https://logiqx.github.io/covid-stats/

## PHE Core

Import library for working with PHE data

In [1]:
import os
import csv

import numpy as np
import matplotlib.pyplot as plt

import common_core
import phe_core

## Area Class

Supports a single area - nation, region, ltla, etc

In [2]:
class Area(common_core.Printable):

    def __init__(self, areaType, areaName):
        """Initialisise the area object"""

        self.data = {}

        self.areaType = areaType
        self.areaName = areaName

        self.safeName = common_core.getSafeName(areaName)
        self.csvName = self.safeName + '.csv'
        

    def load(self, period, dirName):
        """Load demographic data - cases or deaths"""

        fileName = os.path.join(common_core.dataDir, phe_core.PHE_DASHBOARD, "csv", period, dirName, self.areaType, self.csvName)
        partName = common_core.getPartName(fileName)

        try:
            with open(fileName, 'r') as f:
                reader = csv.reader(f, delimiter = ',')

                dtype = []
                converters = {}
                colNames = next(reader)

                for i in range(len(colNames)):
                    colName = colNames[i]
                    if colName == "date":
                        dtype.append((colName, "U10"))
                    elif colName == "areaName":
                        dtype.append((colName, "U40"))
                    elif colName.endswith("Rate"):
                        dtype.append((colName, "f8"))
                        converters[i] = lambda s: float(s or 0)
                    else:
                        dtype.append((colName, "u4"))
                        converters[i] = lambda s: int(s or 0)

                data = np.genfromtxt(f, dtype=dtype, converters=converters, delimiter=",")

            # Ensure period is present in data arrays
            if period not in self.data:
                self.data[period] = {}

            # Store the data
            self.data[period][dirName] = data

        # General catch all
        except:
            print(f"Failed to load {dirName} for {self.areaName}")
            raise


    def loadDaily(self):
        """Load PHE daily data"""

        period = "daily"
        
        if self.areaType != "nhsregion":
            self.load(period, "cases")
            self.load(period, "deaths")
        else:
            self.load(period, "patients")

## Areas Class

Combines multiple areas - nation, region, ltla, etc

In [3]:
class Areas(common_core.Printable):

    def __init__(self, areaType):
        """Initialisise the areas object"""

        self.areaType = areaType
        self.areas = {}

    
    def addArea(self, areaName):
        """Add a new area which can then be loadeded from disk"""
        
        area = Area(self.areaType, areaName)

        self.areas[areaName] = area
        

    def loadArea(self, areaName):
        """Load PHE data for a single area"""
        
        self.areas[areaName].loadDaily()
        

    def plotCases(self):
        """Plot cases by specimen date age demographics"""
        
        # Create new figure
        plt.figure(clear=True, figsize=(16, 9))  
        
        # Plot data
        for areaName in self.areas:
            area = self.areas[areaName]

            # Get all value up to the last non-zero
            tmp_points = np.array(area.data["daily"]["cases"]["cases"])
            idx = np.max(np.nonzero(tmp_points))
            tmp_points = tmp_points[:idx+1-5]

            # Calculate moving average using convolve
            # consider mode = "same" instead of concatenate?
            y_points = np.concatenate((np.zeros(3), np.convolve(tmp_points, np.ones(7)/7, mode="valid")))
            x_points = np.arange(len(y_points))
            x_ticks = np.array(area.data["daily"]["cases"]["date"])[:idx+1]
            
            plt.plot(x_points, y_points, label = area.areaName) #, color='green', linewidth=2)

        # Set xticks
        plt.xticks(np.arange(0, len(x_ticks), step=7), x_ticks[::7], rotation=90)
        
        # Add title, axis labels and legend
        plt.title('Daily Cases Test')
        plt.ylabel('Number of cases')
        plt.legend()
     
        # Show figure
        plt.show()

## Load data into memory

In [4]:
overview = Areas(areaType="overview")
for overviewName in phe_core.overviewNames:
    overview.addArea(overviewName)
    overview.loadArea(overviewName)

regions = Areas(areaType="nation")
for nationName in phe_core.nationNames:
    regions.addArea(nationName)
    regions.loadArea(nationName)

regions = Areas(areaType="region")
for regionName in common_core.regionNames:
    regions.addArea(regionName)
    regions.loadArea(regionName)

nhsRegions = Areas(areaType="nhsregion")
for nhsRegionName in common_core.nhsRegionNames:
    nhsRegions.addArea(nhsRegionName)
    nhsRegions.loadArea(nhsRegionName)

ltlas = Areas(areaType="ltla")
for ltlaName in phe_core.ltlaNames:
    ltlas.addArea(ltlaName)
    ltlas.loadArea(ltlaName)

## Plot Daily Cases

In [5]:
#areas.plotCases()